## Time Series Processing issue
sample notebook to demonstrate issue with initial approach to time series processing 

#### Bring in data for demonstration

In [9]:
from tickerset import TickerSet

In [10]:
#bring in ticker set for analyzing the current processing approach on the ticker time series 
t = TickerSet()

Pulling Ticker data from Yahoo Finance
Creating yf Ticker instance for XLF, fetching history
Creating yf Ticker instance for XHB, fetching history
Creating yf Ticker instance for XLK, fetching history
Creating yf Ticker instance for XLY, fetching history
Creating yf Ticker instance for XLP, fetching history
Creating yf Ticker instance for XRT, fetching history
Creating yf Ticker instance for XLI, fetching history
Creating yf Ticker instance for XLB, fetching history
Creating yf Ticker instance for XTL, fetching history
Creating yf Ticker instance for XLU, fetching history
Creating yf Ticker instance for ^VIX, fetching history


#### Demonstrate Issue "compute price change"

In [11]:
#the columns of interest 
columns = ['ticker','close', 'open', 'prev_close','price_chg_c2o']

In [12]:
#prev close for first row of XHB (and first row of any ticker) is imputed to be current close
#subsequent rows are previous close 
t.df[t.df.ticker.isin(['XLF', 'XHB'])][columns].iloc[5782:5790]


,ticker,close,open,prev_close,price_chg_c2o
5782,XLF,38.67,38.39,38.45,-0.001560
5783,XLF,38.78,38.77,38.67,0.002586
5784,XLF,39.27,39.23,38.78,0.011604
5785,XHB,38.49,39.17,38.49,0.017667
5786,XHB,37.51,38.02,38.49,-0.012211
5787,XHB,38.14,37.45,37.51,-0.001600
5788,XHB,37.86,38.06,38.14,-0.002098
5789,XHB,37.45,37.58,37.86,-0.007396


#### Demonstrate issue with rolling volume 

In [8]:
#volume_rollmed for XHB is computed accross a window including XLF 
columns = ['ticker','volume', "volume_rollmed","volume_diff_to_med","volume_pchg_from_med"]
t.df.iloc[5782:5790][columns]

,ticker,volume,volume_rollmed,volume_diff_to_med,volume_pchg_from_med
5782,XLF,51734400,47958300.0,3776100.0,0.078737
5783,XLF,70424900,47958300.0,22466600.0,0.468461
5784,XLF,71156600,48038950.0,23117650.0,0.481227
5785,XHB,71000,71000.0,0.0,0.000000
5786,XHB,203500,137250.0,66250.0,0.482696
5787,XHB,553500,203500.0,350000.0,1.719902
5788,XHB,70700,137250.0,-66550.0,-0.484882
5789,XHB,264100,203500.0,60600.0,0.297789


In [ ]:
#same idea is true for the rolling volume and bollinger and Z

#To correct for this, we either need to 
# A) calculate the aggregations for each ticker time series separately before joining them together
# B) join them and then compute the aggregations group wise (split apply transform )


#For approach A

#Fetch Ticker, compute time series aggregation
#---- move processing from Ticker Set to Ticker, TickerSet can still union and join with VIX
